In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from coffea import hist
import coffea.processor as processor

In [11]:
fileset = {'tHq': ['root://xcache//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/38E83594-51BD-7D46-B96D-620DD60078A7.root']}

In [12]:
class SignalProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "")
        MET_axis = hist.Bin("MET", "MET pT [GeV]", 18, 0, 90)
        
        self._accumulator = processor.dict_accumulator({'MET': hist.Hist("Events", dataset_axis, MET_axis)})
        self._MET = processor.column_accumulator(np.array([]))
        
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        self._MET += events.MET.pt.flatten()
        return self._MET
    
    def postprocess(self, accumulator):
        return accumulator

In [13]:
output = processor.run_uproot_job(fileset=fileset, 
                       treename="Events", 
                       processor_instance=SignalProcessor(),
                       executor=processor.futures_executor,
                       executor_args={'workers': 8, 'nano': True},
                       chunksize=250000)

ValueError: column_accumulator cannot be added to <class 'numpy.ndarray'>